In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  
from librep.base.estimator import Estimator
%load_ext autoreload
%autoreload 2
from standartized_balanced import StandardizedBalancedDataset
import os,torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_data(dataset_name,sensors,normalize_data):    
    working_directory=f"result/{dataset_name}/"
    data_folder = f"../../../../../../experiment_executor/data/standartized_balanced/{dataset_name}/"
    dataset = StandardizedBalancedDataset(data_folder, sensors=sensors)
    X_train, y_train,X_test, y_test,X_val, y_val = dataset.get_all_data(normalize_data=normalize_data, resize_data=True)
    return X_train, y_train,X_test, y_test,X_val, y_val
        #print(f"shape: X_train {X_train.shape} --- X_test {X_test.shape} --- X_test {X_val.shape}")

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X_train, y_train,X_test, y_test,X_val, y_val=get_data("UCI",['accel','gyro'],False)
input_shape=X_train[0].shape
print(input_shape)

(60, 6)


In [3]:
def train_RandomForestClassifier(simclr_linear):
    data=simclr_linear.transform(X_train)
    data_t=simclr_linear.transform(X_test)
    print(data.shape)
    print(X_train.shape)
    from sklearn.ensemble import RandomForestClassifier
    
    clf = RandomForestClassifier(n_estimators=200)  
    # Entrenar el clasificador
    clf.fit(data, y_train)
    etiquetas_pred = clf.predict(data_t)
    from sklearn.metrics import accuracy_score, classification_report
    accuracy = accuracy_score(y_test, etiquetas_pred)
    report = classification_report(y_test, etiquetas_pred)
    print(accuracy,report)
    return accuracy

In [4]:
#import simclr_utils as sm_ut
#transformations=sm_ut.transform_combinations(3,4)

#print(transformations)

In [5]:
from librep.transforms.simclr_linear import SimCLR_linear

batch_size_head = 256
decay_steps_head = 1000
epochs_head = 300
temperature_head = 0.77
input_shape=X_train[0].shape
n_components=60
print(input_shape)
transform_funcs=[1, 2, 3]
verbose=0
import os,torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#transformations=sm_ut.transform_combinations(3,4)

for transform_funcs in transformations:

    simclr_linear = SimCLR_linear(dataset='motionsense',input_shape=input_shape,
                        n_components=n_components,
                        batch_size_head=batch_size_head,
                        transform_funcs=transform_funcs,
                        temperature_head=temperature_head, 
                        epochs_head=epochs_head,
                        save_model=False,
                        verbose=0,
                              patience=10,
                              min_delta=0.01,
                        total_epochs=50,
                        batch_size=32,
                              device=device,
                        save_reducer=False,
                        lr=0.001)
    print(transform_funcs)
    
    trained_simclr_model = simclr_linear.fit(X_train,y_train)
    train_RandomForestClassifier(simclr_linear)


(60, 6)


NameError: name 'transformations' is not defined

In [ ]:
#print(simclr.model)

In [ ]:
data=simclr_linear.transform(X_train)
data_t=simclr_linear.transform(X_test)
print(data.shape)
print(X_train.shape)
from sklearn.ensemble import RandomForestClassifier

# Crear un clasificador Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)  # Puedes ajustar los hiperparámetros según sea necesario

# Entrenar el clasificador
clf.fit(data, y_train)
etiquetas_pred = clf.predict(data_t)
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, etiquetas_pred)
report = classification_report(y_test, etiquetas_pred)
print(accuracy,report)